# 1. Importación de librerias

In [22]:
import requests
import pickle
import random
import numpy as np
import pandas as pd

import nltk
nltk.download() # Uncomment on first run
import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from sklearn.metrics import classification_report




NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

# 2. Preparación de los datos
En primer lugar se llevó acabo la carga de los archivos. Para esto se iteró por cada una de las categorías en el repositorio dado para el laboratorio usando request HTTP GET para obtener los textos iterativamente, se guardaban los artículos en listas tal que en otra lista en el mismo índice, estaba la categoría de dicho artículo.

In [23]:
"""
Load Articles
"""
categories = ['business', 'entertainment', 'politics', 'sport', 'tech']
articles = []

# Try to load from pickle else download
try:
    articles = pickle.load(open('./articles_raw.p', 'rb'))
except Exception as e:
    print(e)
    for cat in categories:
        cat_articles = []
        i = 1
        f = requests.get('https://raw.githubusercontent.com/jcestevezc/Machine-Learning-Techniques/master/Laboratorio%204/Punto%202/Noticias%20BBC/{}/{}.txt'.format(cat, str(i).zfill(3)))
        while(not f.status_code == 404):
            print(cat, i)
            cat_articles.append(f.text)
            i += 1
            f = requests.get('https://raw.githubusercontent.com/jcestevezc/Machine-Learning-Techniques/master/Laboratorio%204/Punto%202/Noticias%20BBC/{}/{}.txt'.format(cat, str(i).zfill(3)))

        articles.append(cat_articles)

pickle.dump(articles, open('./articles_raw.p', 'wb'))

# Flatten articles array
X_text = []
y = []

print('\n --- Categories ---\n')
for i, cat_articles in enumerate(articles):
    print('{}: {} articles'.format(categories[i], len(cat_articles)))
    for j, article in enumerate(articles[i]):
        X_text.append(article)
        y.append(categories[i])
    
# Shuffle array
zipped = list(zip(X_text, y))
random.shuffle(zipped)
X_text, y = zip(*zipped)

print('\n\n --- Example Article --- \n', X_text[0])


 --- Categories ---

business: 510 articles
entertainment: 386 articles
politics: 417 articles
sport: 511 articles
tech: 401 articles


 --- Example Article --- 
 Bellamy fined after row

Newcastle have fined their Welsh striker Craig Bellamy two weeks' wages - about £80,000 - following his row with manager Graeme Souness.

But Bellamy, 25, has not been put on the transfer list, although he did not train with the first team on Tuesday. Magpies chairman Freddy Shepherd told the Newcastle Evening Chronicle: "It is not about money. It is about a player thinking he is bigger than this club. "No individual is, be it the chairman, the manager or a player." Souness dropped Bellamy for Sunday's game against Arsenal, claiming the Welshman had feigned injury after being asked to play out of position. "When I heard what the manager was saying I was in shock," Bellamy said. "I thought 'not only has he gone behind my back, he's lying'," he said in response to Souness' remarks. And the Wales intern

# 3. Perfilamiento
Se definio el DataFrame que contiene los datos asimilados y se realizaron algunas estadísticas descriptivas junto con el análisis de la herramienta ProfilingReport de Pandas

In [24]:
data = pd.DataFrame(list(zip(X_text, y)), 
               columns =['Article', 'Category']) 
data.head()

,Article,Category
0,Bellamy fined after row\n\nNewcastle have fine...,sport
1,Singer's film to show at festival\n\nA documen...,entertainment
2,Mourinho plots impressive course\n\nChelsea's ...,sport
3,Malik rejects all-black MP lists\n\nA call for...,politics
4,Levy takes Whitbread novel prize\n\nOrange Pri...,entertainment


In [25]:
textos = data.copy()
textos['Conteo'] = [len(x) for x in textos['Article']]
textos['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in textos['Article']]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Article']]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Article']]

ProfileReport(textos)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,6
Number of observations,2225
Total Missing (%),0.0%
Total size in memory,104.4 KiB
Average record size in memory,48.1 B
Numeric,3
Categorical,3
Boolean,0
Date,0
Text (Unique),0
Rejected,0


### 3.1 Limpieza de datos. 
De la misma manera que se explicó en el laboratorio, el preprocesamiento de los datos consistía en 5 aspectos:

1. Remover los caractéres no ASCII.
2. Convertir todas las palabras a minúsculas. 
3. Remover las puntuaciones del texto. 
4. Reemplazar los números.
5. Remover las palabras stopwords.

In [26]:
"""
Preprocessing
"""
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words


def preproccesing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

try:
    X_text = pickle.load(open('./articles_clean.p', 'rb'))
except:
    print('Contractions')
    X_text = list(map(contractions.fix, X_text))
    print('Tokenize')
    X_text = list(map(word_tokenize, X_text))
    print('Preprocessing')
    X_text = list(map(preproccesing, X_text))
    pickle.dump(X_text, open('./articles_clean.p', 'wb'))

### 3.2 Tokenización y limpieza de datos

En este paso creamos la columna que contiene la lista con todas las palabras de un solo artículo


In [27]:
data['Article'] = data['Article'].apply(contractions.fix) #Aplica la corrección de las contracciones
data['Words'] = data['Article'].apply(word_tokenize)
data['Words'] = data['Words'].apply(preproccesing) #Aplica la eliminación del ruido    
data.head()

,Article,Category,Words
0,Bellamy fined after row\n\nNewcastle have fine...,sport,"[bellamy, fined, row, newcastle, fined, welsh,..."
1,Singer's film to show at festival\n\nA documen...,entertainment,"[singer, film, show, festival, documentary, ta..."
2,Mourinho plots impressive course\n\nChelsea's ...,sport,"[mourinho, plots, impressive, course, chelsea,..."
3,Malik rejects all-black MP lists\n\nA call for...,politics,"[malik, rejects, allblack, mp, lists, call, et..."
4,Levy takes Whitbread novel prize\n\nOrange Pri...,entertainment,"[levy, takes, whitbread, novel, prize, orange,..."


### 3.3 Estandarización de los datos
En este paso se lleva acabo la lemmatización y stematización de los verbos así como la eliminación de prefijos y sufijos de las palabras



In [28]:

"""
Stem and Lemmatize
"""

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

try:
    X_text = pickle.load(open('./articles_3.p', 'rb'))
except:
    X_text = list(map(stem_and_lemmatize, X_text))
    X_text = list(map(lambda text : ' '.join(text), X_text))
    pickle.dump(X_text, open('./articles_3.p', 'wb'))

data['Words'] = data['Words'].apply(stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
data['Words'] = data['Words'].apply(lambda x: ' '.join(map(str, x)))
data.head()

,Article,Category,Words
0,Bellamy fined after row\n\nNewcastle have fine...,sport,bellamy fin row newcastl fin welsh striker cra...
1,Singer's film to show at festival\n\nA documen...,entertainment,sing film show fest docu tak candid look lif c...
2,Mourinho plots impressive course\n\nChelsea's ...,sport,mourinho plot impress cours chelse win fulham ...
3,Malik rejects all-black MP lists\n\nA call for...,politics,malik reject allblack mp list cal ethn min sho...
4,Levy takes Whitbread novel prize\n\nOrange Pri...,entertainment,levy tak whitbread novel priz orang priz win a...


# 4. Creación del modelo

### 4.1 Datos de entramiento y prueba

In [30]:
"""
Divide Train Test
"""
X, y = data['Words'],data['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Naive Bayer Multinomial
Posteriormente se pretendía entrenar un clasificador Naive Bayes Multinomial con diferentes parámetros de alpha, la transformación de laplace que ayuda a regularizar los resultados del modelo. 

Se uso un Bayer Multinomial al tener más de una clase objetivo de clasificación, dado que este es el clasificador más adecuado 

In [47]:
"""
Train
"""

estimators = [('transform', CountVectorizer(stop_words='english')), 
              ('clf', MultinomialNB())]
pipe = Pipeline(estimators)
param_grid = dict(transform=[TfidfVectorizer(stop_words='english'), CountVectorizer(stop_words='english'), HashingVectorizer(stop_words='english',n_features=2**13)],
                  clf__alpha=[.01,.1,.5],
                  )
grid_search_NB = GridSearchCV(pipe, param_grid=param_grid,verbose = 20,n_jobs=-1,error_score=0.0)
grid_search_NB.fit(X_train,y_train)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

GridSearchCV(cv=None, error_score=0.0,
             estimator=Pipeline(memory=None,
                                steps=[('transform',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                    

In [48]:
# Parametros del mejor estimador NB Multinomial
grid_search_NB.best_estimator_

Pipeline(memory=None,
         steps=[('transform',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True))],
         verbose=False)

## Ensemblers
Para probar el rendimiento de Naive Bayes se quiso comparar con los mismos clasificadores embebidos vistos en el laboratorio. En particular con 3 modelos principales: 

1. Random Forest
2. BaggingClassifer
3. AdaBoost

In [44]:
estimators = [('transform', CountVectorizer(stop_words='english')), 
              ('clf', BaggingClassifier(base_estimator=SVC(kernel='rbf'), random_state=0))]
pipe = Pipeline(estimators)
param_grid = dict(transform=[TfidfVectorizer(stop_words='english'), CountVectorizer(stop_words='english'), HashingVectorizer(stop_words='english',n_features=2**13)],
                  clf = [RandomForestClassifier(random_state=0),
                         BaggingClassifier(base_estimator=SVC(kernel='rbf'), random_state=0),
                         AdaBoostClassifier(random_state=0)],
                  clf__n_estimators=[10,20,50],
                  )
grid_search_EN = GridSearchCV(pipe, param_grid=param_grid,verbose = 20,n_jobs=-1,error_score=0.0)
grid_search_EN.fit(X_train,y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

GridSearchCV(cv=None, error_score=0.0,
             estimator=Pipeline(memory=None,
                                steps=[('transform',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                    

## 4.2 Mejores clasificadores
Los resultados de mejores clasificadores para ensemblers y para Naive Bayes arrojan los siguientes resultados:

- Ensemblers: Modelo de Bagging Clasifier con un clasificador base basado en una Máquina de vectores de soporte con nucleo gaussiano RBF y 20 estimadores.

- Naive Bayes: Modelo Naive Bayes Multinomial con un valor alpha de transformación de Laplace de 0.5 y los datos transformados a su forma de representación de una matriz con la cuenta de las veces que aparece un término por cada artículo de la BBC

In [55]:
grid_search_NB.best_params_

{'clf__alpha': 0.5,
 'transform': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=1.0, max_features=None, min_df=1,
                 ngram_range=(1, 1), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None)}

In [54]:
grid_search_EN.best_params_

{'clf': BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                      class_weight=None, coef0=0.0,
                                      decision_function_shape='ovr', degree=3,
                                      gamma='scale', kernel='rbf', max_iter=-1,
                                      probability=False, random_state=None,
                                      shrinking=True, tol=0.001, verbose=False),
                   bootstrap=True, bootstrap_features=False, max_features=1.0,
                   max_samples=1.0, n_estimators=20, n_jobs=None,
                   oob_score=False, random_state=0, verbose=0, warm_start=False),
 'clf__n_estimators': 20,
 'transform': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.float64'>, encoding='utf-8',
                 input='content', lowercase=True, max_df=1.0, max_features=None,
                 min_df=1, ngram_range=(1, 1),

# Resultados de los modelos

In [50]:
"""
Hyperparameters for Naive Bayes Multinomial
"""
results = grid_search_NB.cv_results_
resultsSorted = sorted(zip(results["mean_test_score"], results["params"]), key=lambda pair: -pair[0])

print('Best hyperparameters: \n')
for mean_score, params in resultsSorted:
    print('\nmean accuracy: ', mean_score, 'params: ', params)

Best hyperparameters: 


mean accuracy:  0.9712215209221198 params:  {'clf__alpha': 0.5, 'transform': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)}

mean accuracy:  0.9712179245113376 params:  {'clf__alpha': 0.01, 'transform': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=N

In [49]:
"""
Hyperparameters for Ensemblers
"""
results = grid_search_EN.cv_results_
resultsSorted = sorted(zip(results["mean_test_score"], results["params"]), key=lambda pair: -pair[0])

print('Best hyperparameters: \n')
for mean_score, params in resultsSorted:
    print('\nmean accuracy: ', mean_score, 'params: ', params)



Best hyperparameters: 


mean accuracy:  0.966420312528097 params:  {'clf': BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='rbf', max_iter=-1,
                                     probability=False, random_state=None,
                                     shrinking=True, tol=0.001, verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=20, n_jobs=None,
                  oob_score=False, random_state=0, verbose=0, warm_start=False), 'clf__n_estimators': 20, 'transform': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_featu

In [52]:
"""
Validation
"""
predictions_NB = grid_search_NB.predict(X_test)
predictions_EN = grid_search_EN.predict(X_test)

print('__ Test Scores for NB __\n')
print(classification_report(y_test, predictions_NB, target_names=['business', 'entertainment', 'politics', 'sport', 'tech']))

print('__ Test Scores for Ensemblers__\n')
print(classification_report(y_test, predictions_EN, target_names=['business', 'entertainment', 'politics', 'sport', 'tech']))


__ Test Scores for NB __

               precision    recall  f1-score   support

     business       1.00      0.98      0.99       114
entertainment       1.00      0.96      0.98        98
     politics       0.99      1.00      1.00       101
        sport       1.00      1.00      1.00       133
         tech       0.96      1.00      0.98       111

     accuracy                           0.99       557
    macro avg       0.99      0.99      0.99       557
 weighted avg       0.99      0.99      0.99       557

__ Test Scores for Ensemblers__

               precision    recall  f1-score   support

     business       0.96      0.97      0.97       114
entertainment       0.99      0.94      0.96        98
     politics       0.96      0.99      0.98       101
        sport       0.99      1.00      0.99       133
         tech       1.00      0.98      0.99       111

     accuracy                           0.98       557
    macro avg       0.98      0.98      0.98       557
 

## 5.1 Conclusiones de resultados
Se concluye nuevamente que para tareas de clasificación relativamente sencillas y particularmente para procesamiento de textos, el clasificador Naive Bayes tiene un rendimiento muy por encima que otros clasificadores más compuestos. Esto se concluye debido a que a pesar de que los resultados han sido relativamente los mismo, entrenar un modelo BaggingClassifier como el obtenido, sobretodo con un clasificador base como una SVM tiene un tiempo de entrenamiento mucho más largo y costoso que un modelo NB Multinomial. 

Adicionalmente se puede observar que para el modelo NB: 
- Las metricas de Recall son más altas por lo que se está dejando de clasificar correctamente menos que en el modelo Ensembler

- No hay signos de sobreajuste, y el modelo parece casi perfecto en términos del trade-off Recall/Precision simbolizado en el coeficiente F1